pascalvoc: resize original images

Strategy:
1. Read: Images RGB / Masks RGB (original dataset)
2. Pad (512,512) for training
3. Save: Images RGB / Masks RGB

In [ ]:
padsize = (512,512)

environment

In [ ]:
using Pkg
envpath = expanduser("~/envs/dev/")
Pkg.activate(envpath)

In [12]:
# libs
using Images
using DataFrames
using CSV
using FLoops

In [ ]:
function pad_image(img, newsize)
    if any(size(img) .> newsize)   error("newsize must be larger than img")   end
    pad = ((newsize[1] - size(img, 1)) ÷ 2, (newsize[2] - size(img, 2)) ÷ 2)
    return Images.PaddedView(0, img, newsize, pad)
end

In [ ]:
# input folders
datasetPath = "~/datasets/pascalvoc/VOCdevkit/VOC2012/"
imagesPath  = datasetPath * "JPEGImages/"
masksPath   = datasetPath * "SegmentationClass/"
classlist   = datasetPath * "ImageSets/Main/cow_trainval.txt"

# output folders
workpath = pwd() * "/"
workpath = replace(workpath, homedir() => "~")
resizedImagesPath = workpath * "dataset/images/"
resizedMasksPath  = workpath * "dataset/masks/"
mkpath(expanduser(resizedImagesPath))
mkpath(expanduser(resizedMasksPath))

input dataset

In [15]:
fpfn = expanduser(workpath) * "dftrain.csv"
dftrain = CSV.read(fpfn, DataFrame)
fpfn = expanduser(workpath) * "dfvalid.csv"
dfvalid = CSV.read(fpfn, DataFrame)
fpfn = expanduser(workpath) * "dftest.csv"
dftest = CSV.read(fpfn, DataFrame);

resize dataset

In [16]:
# resize images and masks
dfs   = [dftrain, dfvalid, dftest]

for df in dfs
@floop for row in eachrow(df)
    # images
    fpfn = expanduser(row.X)
    img = Images.load(fpfn)                          # read
    img = pad_image(img, padsize) .|> RGB            # pad
    fpfn = expanduser(resizedImagesPath) * basename(fpfn)
    Images.save(fpfn, img)                           # save

    # masks
    fpfn = expanduser(row.y)
    mask = Images.load(fpfn)                     # read
    mask = pad_image(mask, padsize) .|> RGB      # pad
    fpfn = expanduser(resizedMasksPath) * basename(fpfn)
    Images.save(fpfn, mask)                      # save
end
end

In [ ]:
# create data frames
dftrain_resized = dftrain
dftrain_resized.X = @. resizedImagesPath * basename(dftrain.X)
dftrain_resized.y = @. resizedMasksPath  * basename(dftrain.y)

dfvalid_resized = dfvalid
dfvalid_resized.X = @. resizedImagesPath * basename(dfvalid.X)
dfvalid_resized.y = @. resizedMasksPath  * basename(dfvalid.y)

dftest_resized = dftest
dftest_resized.X = @. resizedImagesPath * basename(dftest.X)
dftest_resized.y = @. resizedMasksPath  * basename(dftest.y)

# save data frames
fpfn = expanduser(workpath) * "dftrain-resized.csv"
CSV.write(fpfn, dftrain_resized)
fpfn = expanduser(workpath) * "dfvalid-resized.csv"
CSV.write(fpfn, dfvalid_resized)
fpfn = expanduser(workpath) * "dftest-resized.csv"
CSV.write(fpfn, dftest_resized)

In [ ]:
# checkpoint
df = CSV.read(expanduser(workpath) * "dftrain-resized.csv", DataFrame)

k = rand(1:size(df, 1))
img = Images.load(expanduser(df.X[k]))
msk = Images.load(expanduser(df.y[k]))
mosaicview([img, msk]; nrow=1)